# Analyze dDEMs as a function of land cover type, canopy height, and terrain parameters

In [ ]:
import os
import matplotlib.pyplot as plt
import rioxarray as rxr
import xarray as xr
import numpy as np
import xdem
import seaborn as sns
import pandas as pd

In [ ]:
# pipeline inputs
data_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/'
refdem_fn = os.path.join(data_dir, 'study-sites', 'MCS', 'refdem', 'MCS_REFDEM_WGS84.tif')
chm_fn = os.path.join(data_dir, 'study-sites', 'MCS', 'refdem', 'chm_mcs_1m.tif')

# pipeline outputs
job_dir = os.path.join(data_dir, 'snow_depth_maps', 'MCS_20240420-1_reftrees_sourcetrees')
ddem_fn = os.path.join(job_dir, 'coreg_final_diff', 'ddem.tif')
roads_mask_fn = os.path.join(job_dir, 'land_cover_masks', 'roads_mask.tif')
snow_mask_fn = os.path.join(job_dir, 'land_cover_masks', 'snow_mask.tif')
ss_mask_fn = os.path.join(job_dir, 'land_cover_masks', 'stable_surfaces_mask.tif')
trees_mask_fn = os.path.join(job_dir, 'land_cover_masks', 'trees_mask.tif')

# output folder
out_dir = os.path.join(job_dir, 'ddem_analysis')
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    print('Made directory for outputs:', out_dir)

## Load reference DEM

In [ ]:
# Reference DEM
refdem = rxr.open_rasterio(refdem_fn).squeeze()

# Functioin to load and adjust other files
def load_adjust_file(fn, refdem):
    file = rxr.open_rasterio(fn).squeeze()
    crs = file.rio.crs
    file = xr.where(file==file.attrs['_FillValue'], np.nan, file)
    file = file.sel(x=refdem.x.data, y=refdem.y.data, method='nearest')
    file = file.rio.write_crs(crs)
    return file


## dDEM vs. canopy height

In [ ]:
# Load input files
ddem = load_adjust_file(ddem_fn, refdem)
chm = load_adjust_file(chm_fn, refdem)

# Create pandas.DataFrame
# df = pd.DataFrame({'dDEM': np.ravel(ddem.data),
#                    'CHM': np.ravel(chm.data)})
# df.dropna(inplace=True)
# df.reset_index(drop=True, inplace=True)
# df.loc[df['CHM'] < 0, 'CHM'] = 0

# # Create bins for CHM
# df['CHM_bin'] = pd.cut(df['CHM'], bins=np.linspace(-1,50,52))

# # Plot
# fig, ax = plt.subplots(figsize=(14,8))
# sns.boxplot(data=df, x='CHM_bin', y='dDEM', showfliers=False, ax=ax)
# ax.set_xticks(ax.get_xticks())
# ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
# ax.set_ylabel('dDEM [m]')
# ax.set_xlabel('Canopy height [m]')
# plt.show()

# # Save figure
# fig_fn = os.path.join(out_dir, 'ddem_vs_chm.png')
# fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
# print('Figure saved to file:', fig_fn)

## Add the canopy height model back...

In [ ]:
ddem_adj = ddem.copy()
ddem_adj.data = ddem_adj.data + chm.data

fig, ax = plt.subplots(1, 2, figsize=(10,6))
ax[0].imshow(ddem.data, cmap='coolwarm_r', clim=(-5, 5),
             extent=(np.min(ddem.x.data)/1e3, np.max(ddem.x.data)/1e3,
                     np.min(ddem.y.data)/1e3, np.max(ddem.y.data)/1e3))
ax[0].set_title('dDEM')
im = ax[1].imshow(ddem_adj.data, cmap='coolwarm_r', clim=(-5, 5),
                  extent=(np.min(ddem.x.data)/1e3, np.max(ddem.x.data)/1e3,
                          np.min(ddem.y.data)/1e3, np.max(ddem.y.data)/1e3))
ax[1].set_title('dDEM + CHM')
fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.9, 0.2, 0.025, 0.6])
fig.colorbar(im, cax=cbar_ax, label='Elevation difference [m]')
plt.show()

## dDEM vs. land cover type

In [ ]:
# Original dDEM
# Land cover masks
roads_mask = load_adjust_file(roads_mask_fn, refdem)
snow_mask = load_adjust_file(snow_mask_fn, refdem)
ss_mask = load_adjust_file(ss_mask_fn, refdem)
trees_mask = load_adjust_file(trees_mask_fn, refdem)

# Create dataframe
df = pd.DataFrame({'dDEM': np.ravel(ddem.data),
                   'roads_mask': np.ravel(roads_mask.data),
                   'snow_mask': np.ravel(snow_mask.data),
                   'ss_mask': np.ravel(ss_mask.data),
                   'trees_mask': np.ravel(trees_mask.data)})
df.dropna(inplace=True)

# Plot histogram
bins = np.linspace(-10, 10, 200)
cols = ['snow_mask', 'ss_mask', 'trees_mask', 'roads_mask'] 
colors = [(55/255, 126/255, 184/255, 1), # snow
          (153/255, 153/255, 153/255, 1), # stable surfaces
          (77/255, 175/255, 74/255, 1), # trees
          (166/255, 86/255, 40/255, 1)] # roads
fig, ax = plt.subplots(2, 2, figsize=(10,5))
ax = ax.flatten()
i=0
for col, color in zip(cols, colors):
    df_sub = df.loc[df[col]==1, 'dDEM']
    ax[i].hist(df_sub.values, bins=bins, color=color, alpha=0.8, label=col)
    ax[i].set_xlim(-5, 5)
    ax[i].set_title(col.replace('ss_mask', 'stable surfaces').replace('_mask', ''))
    i+=1
    
fig.tight_layout()
plt.show()

# Save figure
fig_fn = os.path.join(out_dir, 'ddem_vs_land_cover_types.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

In [ ]:
# Adjusted dDEM
# Create dataframe
df = pd.DataFrame({'dDEM_adj': np.ravel(ddem_adj.data),
                   'roads_mask': np.ravel(roads_mask.data),
                   'snow_mask': np.ravel(snow_mask.data),
                   'ss_mask': np.ravel(ss_mask.data),
                   'trees_mask': np.ravel(trees_mask.data)})
df.dropna(inplace=True)

# Plot histogram
bins = np.linspace(-10, 10, 200)
cols = ['snow_mask', 'ss_mask', 'trees_mask', 'roads_mask'] 
colors = [(55/255, 126/255, 184/255, 1), # snow
          (153/255, 153/255, 153/255, 1), # stable surfaces
          (77/255, 175/255, 74/255, 1), # trees
          (166/255, 86/255, 40/255, 1)] # roads
fig, ax = plt.subplots(2, 2, figsize=(10,5))
ax = ax.flatten()
i=0
for col, color in zip(cols, colors):
    df_sub = df.loc[df[col]==1, 'dDEM_adj']
    ax[i].hist(df_sub.values, bins=bins, color=color, alpha=0.8, label=col)
    ax[i].set_xlim(-5, 5)
    ax[i].set_title(col.replace('ss_mask', 'stable surfaces').replace('_mask', ''))
    i+=1
    
fig.tight_layout()
plt.show()

# Save figure
fig_fn = os.path.join(out_dir, 'ddem+chm_vs_land_cover_types.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

## dDEM vs. terrain parameters